In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from nltk.corpus import stopwords
from textblob import TextBlob
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from nltk.stem import PorterStemmer
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from scipy.sparse import hstack
import matplotlib.pyplot as plt
from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve

In [2]:
# Step 1: Read the CSV file into a pandas DataFrame
combined_df = pd.read_csv("journal_merged.csv")

# Step 2: Rename the column "Introduction" to "Intro"
combined_df.rename(columns={"Introduction": "Intro"}, inplace=True)

# Step 3: Drop rows where the "Intro" column has NaN values
combined_df.dropna(subset=['Intro'], inplace=True)

# Save the modified dataframe to a new CSV if needed
# df.to_csv("/path_to_save/new_journal_merged.csv", index=False)

In [3]:
# Assuming you have already loaded combined_df somewhere above in your code

# Make a copy of the original dataframe to work on
df_cleaned = combined_df.copy()

# Set up the stop words
stop_words = set(stopwords.words('english'))

# Function to clean the text
def clean_text(text):
    # Remove content between parentheses using regex
    text = re.sub(r'\([^)]*\)', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove HTML tags/markup
    text = re.sub(r'<.*?>', '', text)
    # Remove punctuations, numbers and other non-alphabetic characters
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Replace multiple spaces with a single space
    text = re.sub('\s+', ' ', text).strip()
    return text

# Function to remove stop words
def remove_stop_words(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

# Function for lemmatization
def lemmatize_text(text):
    wnl = WordNetLemmatizer()
    tokens = word_tokenize(text)
    lemmatized_tokens = [wnl.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

# Process tracking counter
counter = 0

# Apply the cleaning function
def process_and_track(func, text):
    global counter
    counter += 1
    if counter % 10000 == 0:
        print(f"Processed {counter} records for cleaning.")
    return func(text)

# Apply the cleaning function
df_cleaned['Intro_1'] = df_cleaned['Intro'].apply(lambda x: process_and_track(clean_text, x))
df_cleaned['Intro_Cleaned'] = df_cleaned['Intro'].apply(lambda x: process_and_track(clean_text, x))
# Apply removal of stop words
df_cleaned['Intro_2'] = df_cleaned['Intro'].apply(lambda x: process_and_track(remove_stop_words, x))
df_cleaned['Intro_Cleaned'] = df_cleaned['Intro_Cleaned'].apply(lambda x: process_and_track(remove_stop_words, x))
df_cleaned['Intro_3'] = df_cleaned['Intro_1'].apply(lambda x: process_and_track(remove_stop_words, x))
# Apply lemmatization
df_cleaned['Intro_Cleaned'] = df_cleaned['Intro_Cleaned'].apply(lambda x: process_and_track(lemmatize_text, x))

# Save cleaned version
df_cleaned.to_csv('data_cleaned.csv', index=False)

Processed 10000 records for cleaning.
Processed 20000 records for cleaning.
Processed 30000 records for cleaning.
Processed 40000 records for cleaning.
Processed 50000 records for cleaning.
Processed 60000 records for cleaning.
Processed 70000 records for cleaning.
Processed 80000 records for cleaning.
